# Weapons & Circumstances cosine similarity for Female Victims

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from mlxtend.preprocessing import minmax_scaling

from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity

import pickle as pk

from jupyterthemes import jtplot
jtplot.style(figsize=(15, 9))

## Data

In [2]:
map_data = pd.read_csv('../data/SHR65_22.csv', index_col=0)
map_data.head()

,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,Month,Incident,...,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,MSA
ID,,,,,,,,,,,,,,,,,,,,,
197601001AKASP00,"Juneau, AK",AKASP00,Alaska,State Troopers,Primary state LE,FBI,Yes,1976,January,1,...,American Indian or Alaskan Native,Unknown or not reported,Knife or cutting instrument,Husband,Other arguments,NaN,0,0,30180.0,Rural Alaska
197601001AL00102,"Jefferson, AL",AL00102,Alabama,Birmingham,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Shotgun,Acquaintance,Felon killed by private citizen,Felon killed in commission of a crime,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00104,"Jefferson, AL",AL00104,Alabama,Fairfield,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Shotgun,Wife,Other,NaN,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00106,"Jefferson, AL",AL00106,Alabama,Leeds,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Knife or cutting instrument,Brother,Other arguments,NaN,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00201,"Mobile, AL",AL00201,Alabama,Mobile,Municipal police,FBI,Yes,1976,January,1,...,White,Unknown or not reported,Strangulation - hanging,Acquaintance,Circumstances undetermined,NaN,0,0,30180.0,"Mobile, AL"


In [6]:
outliers = [
    {
        'name': 'Strangulation - hanging',
        'Texas': [1976, 1981],
        'Maryland': [1992, 1998],
        'California': [[1976, 1987], [2013, 2020]],
        'Oregon': [1997, 2003]
    },
    {
        'name': 'Fire',
        'Pennsylvania': [1976, 1990],
        'New York': [[1976, 1989], [1992, 1996]],
        'New Jersey': [1979, 1990],
        'California': [1989, 1998]
    },
    {
        'name': 'Other gun',
        'North Carolina': [1998, 2018]
    }
]

In [4]:
map_data['Weapon'].unique()

array(['Knife or cutting instrument', 'Shotgun',
       'Strangulation - hanging', 'Rifle',
       'Handgun - pistol, revolver, etc',
       'Personal weapons, includes beating', 'Fire',
       'Other or type unknown', 'Firearm, type not stated',
       'Asphyxiation - includes death by gas',
       'Narcotics or drugs, sleeping pills',
       'Blunt object - hammer, club, etc', 'Drowning',
       'Pushed or thrown out window', 'Other gun',
       'Poison - does not include gas', 'Explosives',
       'Weapon Not Reported'], dtype=object)

In [13]:
df = map_data[(map_data['Weapon']=='Strangulation - hanging')
              & (map_data['State']=='Texas')
              & (map_data['VicSex']=='Female')
              & (map_data['Year'].between(1976, 1981))]
df.head()

,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,Month,Incident,...,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,MSA
ID,,,,,,,,,,,,,,,,,,,,,
197602001TX03400,"Cass, TX",TX03400,Texas,Cass County,Sheriff,FBI,Yes,1976,February,1,...,Black,Unknown or not reported,Strangulation - hanging,Other - known to victim,Other,NaN,0,0,30180.0,Rural Texas
197602001TX03401,"Cass, TX",TX03401,Texas,Atlanta,Municipal police,FBI,Yes,1976,February,1,...,Black,Unknown or not reported,Strangulation - hanging,Acquaintance,Other,NaN,0,0,30180.0,Rural Texas
197603015TXDPD00,"Dallas, TX",TXDPD00,Texas,Dallas,Municipal police,FBI,No,1976,March,15,...,Unknown,Unknown or not reported,Strangulation - hanging,Relationship not determined,Other,NaN,0,0,30180.0,"Dallas-Fort Worth-Arlington, TX"
197604019TXHPD00,"Harris, TX",TXHPD00,Texas,Houston,Municipal police,FBI,Yes,1976,April,19,...,White,Unknown or not reported,Strangulation - hanging,Daughter,Other,NaN,0,0,30180.0,"Houston-Sugar Land-Baytown, TX"
197606001TX23501,"Victoria, TX",TX23501,Texas,Victoria,Municipal police,FBI,No,1976,June,1,...,Unknown,Unknown or not reported,Strangulation - hanging,Relationship not determined,Other,NaN,0,0,30180.0,"Victoria, TX"


In [14]:
df.shape

(90, 29)

In [21]:
norm = map_data[(map_data['Weapon']=='Strangulation - hanging')
                & (map_data['State']=='Texas')
                & (map_data['VicSex']=='Female')]

norm.groupby('Solved').mean()

,Year,Incident,VicAge,OffAge,VicCount,OffCount,FileDate
Solved,,,,,,,
No,1991.679443,7.648084,92.547038,999.000000,0.041812,0.000000,55501.344948
Yes,1996.338415,5.189024,48.923780,72.362805,0.094512,0.176829,58551.509146


In [26]:
df.groupby('Solved').count()['Ori'] / 5

Solved
No     10.2
Yes     7.8
Name: Ori, dtype: float64